<a href="https://colab.research.google.com/github/GEOCOMP-Brucellosis-Project/Project-Repo/blob/master/Brucellosis_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Earth Engine

The following code is used to aggregate environmental parameters by month-by county using Google Earth Engine.

First things first, Authenticate EarthEngine with an earthengine account. This will only need to be done the first time you run this script.

Run **only one** of the following code cells: 

1) Use my earthengine credentials to run the code

In [0]:
#creates a credentials file in your content folder
f=open('credentials', 'w+')
f.write('{"refresh_token": "1//053DTDEe3wnC8CgYIARAAGAUSNwF-L9IrZ9ykjLejw7SDXL-JCzuisH0d7EuscyNK0-JI-kgoC8Xp-b4hj1N63Dzfu8EVb34c-rI"}')
f.close()

#copies the credentials file to where it needs to be (! indicates command line statement)
!sudo cp '/content/credentials' '/root/.config/earthengine/credentials'

2) Use your own credentials (requires you apply for an earthengine account)

In [0]:
import ee
ee.Authenticate()

## Code

### Imports

In [0]:
import ee
import pandas
ee.Initialize()

### Functions

In [0]:
def imageParams(image, shapefile):
    """
    Takes an image and a shapefile, returns a dataframe with the mean band values summarized by shapefile features. 
    """
    #Gets the average of the band values in each feature in the iran asset.
    envFC=image.reduceRegions(shapefile, ee.Reducer.mean()) #returns a feature collection
    envDict=envFC.getInfo()['features'] #reads the feature collection into a python ready dictionary. Slow. 

    #EnvDict is a messy list of dictionaries for each feature. We need to extract the "properties" from each feature. 
    cols=[] 
    for feature in envDict:
        cols.append(feature['properties']) #properties contains the data we want
    
    return pandas.DataFrame(cols) #creates a new pandas dataframe with the data in cols
    


def getYearlyParams(collection, shapefile, startYear, endYear):
    """
    Returns the mean of each band in collection aggregated by region in shapefile for each month between startYear and endYear.
    -collection: an EarthEngine ImageCollection filtered to only the desired bands 
    -shapefile: an EarthEngine FeatureColleciton with regions to be summarized (must be named ADM2_EN)
    -startYear, endYear: The code will summarize these years, January-December
    """
    yearlyParams=pandas.DataFrame(columns=['ADM2_EN']) #sets up an output dataframe
    for year in range(startYear, endYear+1):
        print(year)
        for month in range(1, 13):
            print(month, end=' ')
            #Filters the Image Collection object by year, and then by month, and then takes the first image in the resulting collection. The result is an Image object.
            image=collection.filter(ee.Filter.calendarRange(year,year,'year'))\
                .filter(ee.Filter.calendarRange(month, month,'month'))\
                .first()
            #Gets a dataframe with the desired parameters for the monthly image and adds it to the total dataframe with a _YearMM suffix
            yearlyParams=yearlyParams.merge(imageParams(image, shapefile),\
                                            how='outer',\
                                            on='ADM2_EN',\
                                            suffixes=('', '_'+str(year)+str(month).zfill(2)))
    print("done")
    return yearlyParams

### Main

In [0]:
moWeather = ee.ImageCollection("ECMWF/ERA5/MONTHLY").select("mean_2m_air_temperature","total_precipitation") #Loads the ERA5 monthly collection, only two bands
iran = ee.FeatureCollection("users/tannerjohnson56/iran_admin").select("ADM2_EN") #Loads the iran boundaries asset with only county name field kept


yearlyParams=getYearlyParams(moWeather, iran, 2008, 2008) #Gets the yearly parameters

yearlyParams.to_csv(r'yearlyParams.csv',index = False) #Write the results to a .csv

2008
1 2 3 4 5 6 7 8 9 10 11 12 done
